In [2]:
!pip install pampy

In [3]:
from pampy import match, _

In [4]:
def fib(n):
    return match(n,
         1, 1,
         2, 1,
         _, lambda x: fib(x - 1) + fib(x - 2))

In [7]:
[fib(i) for i in range(1, 15)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]